# Sparse

PREPARE for the sparse chemistry Hamiltonian in second quantization.

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `PrepareSparse`
Prepare oracle for the sparse chemistry Hamiltonian

Prepare the state:

$$
    |0\rangle|+\rangle|0\rangle|0\rangle
    \sum_{\sigma}\sum_{pq}
    \sqrt{\frac{T_{pq}'}{2\lambda}}
    |\theta_{pq}^T\rangle|pq\sigma\rangle|000\rangle
    +|1\rangle|+\rangle|+\rangle|+\rangle
    \sum_{\alpha\beta}
    \sum_{pqrs}
    \sqrt{\frac{\tilde{V}_{pqrs}'}{2\lambda}}
    |\theta_{pqrs}^V\rangle|pq\alpha\rangle|rs\beta\rangle
$$

#### Parameters
 - `num_spin_orb`: The number of spin orbitals.
 - `num_non_zero`: The number of non-zero matrix elements.
 - `num_bits_state_prep`: the number of bits of precision for state preparation. This will control the size of the keep register.
 - `num_bits_rot_aa`: The number of bits of precision for the single-qubit rotation for amplitude amplification during the uniform state preparataion. Default 8.
 - `adjoint`: Whether we are apply PREPARE or PREPARE^dag
 - `k`: qroam blocking factor. 

#### Registers
 - `pqrs`: the register to store the spatial orbital index.
 - `theta`: sign qubit.
 - `alpha`: spin for (pq) indicies.
 - `beta`: spin for (rs) indicies.
 - `swap_pq`: a |+> state to restore the symmetries of the p and q indices.
 - `swap_rs`: a |+> state to restore the symmetries of the r and s indices.
 - `swap_pqrs`: a |+> state to restore the symmetries of between (pq) and (rs).
 - `flag_1b`: a single qubit to flag whether the one-body Hamiltonian is to be applied or not during SELECT. 

Refererences:
    [Even More Efficient Quantum Computations of Chemistry Through Tensor
        hypercontraction](https://arxiv.org/abs/2011.03494) Eq. A11.

In [ ]:
from qualtran.bloqs.chemistry.sparse.prepare import PrepareSparse

### Example Instances

In [ ]:
num_spin_orb = 4
tpq = np.random.random((num_spin_orb//2, num_spin_orb//2))
tpq = 0.5 * (tpq + tpq.T)
eris = np.random.random((num_spin_orb//2,) * 4)
eris += np.transpose(eris, (0, 1, 3, 2))
eris += np.transpose(eris, (1, 0, 2, 3))
eris += np.transpose(eris, (2, 3, 0, 1))
prep = PrepareSparse.from_hamiltonian_coeffs(num_spin_orb, tpq, eris)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([prepare_sparse],
           ['`prepare_sparse`'])

### Call Graph

In [ ]:
prepare_sparse_g, prepare_sparse_sigma = prepare_sparse.call_graph()
show_call_graph(prepare_sparse_g)
show_counts_sigma(prepare_sparse_sigma)